In [2]:
import pandas as pd
import numpy as np
     
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

from nltk.tokenize import word_tokenize 

import nltk
nltk.download("punkt")
from nltk.corpus import stopwords
from collections import Counter
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import warnings
warnings.filterwarnings("ignore")

from os import path

accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'
print(accelerator)
import csv
from google.colab import drive
drive.mount('/content/drive')



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


cu80
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Data Reading

In [27]:
olid_training=pd.read_csv("/content/drive/MyDrive/OLIDv1.0/olid-training-v1.0.tsv",sep="\t")
X_train_FULL=olid_training[["id","tweet","subtask_a"]] 
Y_train_FULL=olid_training["subtask_a"]
X_test_FULL=pd.read_csv('/content/drive/MyDrive/OLIDv1.0/testset-levela.tsv',sep='\t',encoding='utf8',quoting=csv.QUOTE_NONE)
Y_TEST=pd.read_csv('/content/drive/MyDrive/OLIDv1.0/labels-levela.csv',sep=',',encoding='utf8',quoting=csv.QUOTE_NONE,header=None)
Y_TRAIN_ENCODED_FULL=[1 if i ==  'OFF' else 0 for i in Y_train_FULL]
Y_TEST_ENCODED_FULL = [1 if i ==  'OFF' else 0 for i in Y_TEST[1]]

## Pre-processing

In [28]:
"""
	 Converting all text to lowercase 
	 Removing all "\#" symbols
	 Removing all punctuation
	 Removing @user tokens
"""
X_train_FULL.tweet = X_train_FULL.tweet.str.lower()

filtered_tweets=[]
for tweet in X_train_FULL["tweet"]:
    tweet_tokens = word_tokenize(tweet) 

    filtered_sentence = [w for w in tweet_tokens if (( w!='user' and w!=':' and w!='url' and w!='@' and w!=',' and w!= "'" and w!='.'and w!='#' and w!='?'))] 
      
    filtered_tweets.append(filtered_sentence)

X_train_FULL["tweet_initial"]=filtered_tweets


X_test_FULL.tweet = X_test_FULL.tweet.str.lower()

filtered_tweets=[]
for tweet in X_test_FULL["tweet"]:
    tweet_tokens = word_tokenize(tweet) 

    filtered_sentence = [w for w in tweet_tokens if (( w!='user' and w!=':' and w!='url' and w!='@' and w!=',' and w!= "'" and w!='.'and w!='#' and w!='?'))] 
      
    filtered_tweets.append(filtered_sentence)

X_test_FULL["tweet_initial"]=filtered_tweets




#FOR TEST

z=[]
for tweet in X_train_FULL["tweet_initial"]:
    d=" ".join(tweet)
    z.append(d)
X_train_FULL["tweet_initial_nontoken"]=z



#FOR TEST

z=[]
for tweet in X_test_FULL["tweet_initial"]:
    d=" ".join(tweet)
    z.append(d)
X_test_FULL["tweet_initial_nontoken"]=z



In [25]:
from tensorflow.keras import backend as K
from tensorflow.keras import initializers, regularizers, constraints
from tensorflow.keras.layers import Layer
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
from tensorflow.keras.layers import Embedding
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
def recall_m(true_Y, pred_Y):
        TP = K.sum(K.round(K.clip(true_Y * pred_Y, 0, 1)))
        possible_pos = K.sum(K.round(K.clip(true_Y, 0, 1)))
        rec = TP / (possible_pos + K.epsilon())
        return rec

def precision_m(true_Y, pred_Y):
        true_positives = K.sum(K.round(K.clip(true_Y * pred_Y, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(pred_Y, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision


    

def f1_m(true_Y, pred_Y):
    pres = precision_m(true_Y, pred_Y)
    rec = recall_m(true_Y, pred_Y)
    return 2*((pres*rec)/(pres+rec+K.epsilon()))

from tensorflow.keras.models import Sequential
from tensorflow.keras import layers


## Tweeter Word2vec /

In [7]:
fname= "/content/drive/MyDrive/GoogleNews-vectors-negative300.bin.gz"
import gensim
from gensim.models import Word2Vec
word_vectors = gensim.models.KeyedVectors.load_word2vec_format(fname, binary=True)  # you can load this saved keyedvectors model later



## Public Word2vec


In [ ]:

#fname='/content/drive/My Drive/Public-Embeddings/word2vec/haber-P1_S1_L0-100V_5E_1547557988.bin.wv.vectors.npy'
fname= "/content/drive/My Drive/Public-Embeddings/word2vec/haber-P1_S1_L0-100V_5E_1547557988.bin"

#fname= '/content/drive/My Drive/OFFENSEVAL20-DATA/haber-P0_S1_L0-100V_5E_v2v.bin'
import gensim
from gensim.models import Word2Vec



#word_vectors = gensim.models.KeyedVectors.load_word2vec_format(fname,encoding='utf8',binary=True)  # you can load this saved keyedvectors model later

#word_vectors=gensim.models.Word2Vec.load(fname)

word_vectors=gensim.models.KeyedVectors.load(fname)
#models = gensim.models.Word2Vec.load(fname)

#gensim.models.KeyedVectors.load_word2vec_format(fname,encoding='utf8')


UnpicklingError: ignored

## Tweeter Fasttext


In [ ]:
import gensim
from gensim.models import FastText

word_vectors = gensim.models.FastText.load_fasttext_format('/content/drive/My Drive/Twitter/FastText/fastText_25022020.bin',encoding='utf-8') # use that if you want to use fasttedxt 



## Public FastText

In [ ]:
word_vectors = gensim.models.FastText.load_fasttext_format('/content/drive/My Drive/OFFENSEVAL20-DATA/haber-P1_S0_L0.bin',encoding='utf-8') # use that if you want to use fasttedxt 


## Tokenizing / creating vocabulary and wordindex using keras functinalities


In [29]:
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words=98790)  #the maximum number of words to keep, based on word frequency. Only the most common num_words-1 words will be kept.
tokenizer.fit_on_texts(X_train_FULL["tweet_initial_nontoken"])
X_train_initial = tokenizer.texts_to_sequences(X_train_FULL["tweet_initial_nontoken"])
X_test_initial = tokenizer.texts_to_sequences(X_test_FULL["tweet_initial_nontoken"])
vocab_size_initial = len(tokenizer.word_index) + 1 
wordIndex_initial=tokenizer.word_index # it is  index
from keras_preprocessing.sequence import pad_sequences
max_len = 50

"""
Padding

"""
X_train_initial = pad_sequences(X_train_initial, padding='post', maxlen=max_len)
X_test_initial = pad_sequences(X_test_initial, padding='post', maxlen=max_len)




In [24]:
# 测试清洗后的数据
X_train_FULL = np.array(X_train_FULL)
X_test_FULL = np.array(X_test_FULL)
Y_TRAIN_ENCODED_FULL = np.array(Y_TRAIN_ENCODED_FULL)
Y_TEST_ENCODED_FULL = np.array(Y_TEST_ENCODED_FULL)

Custom Embedding Layer

In [30]:
def createEmbeddingLayer(wordIndex,not_static):
  a=[]
  embedding_dim=300
  vocabulary_size=len(wordIndex)+1
  embedding_matrix = np.zeros((vocabulary_size, embedding_dim))
  missed=0
  for word, i in wordIndex.items():
    
          
      try:
          embedding_vector = word_vectors[word] # or fast text
          embedding_matrix[i] = embedding_vector
            

      except KeyError: # If word is not found in the word2vec vocabulary , assign random weights
        embedding_matrix[i]=np.random.normal(0,np.sqrt(0.25),embedding_dim)
        missed+=1
        a.append(word)

  print('missed_words :' , missed)

  custom_embedding_layer = Embedding(vocabulary_size,
                                embedding_dim,
                                weights=[embedding_matrix],
                                trainable=not_static )# Controls the updating weights )
  return custom_embedding_layer

  

## BiLSTM

In [31]:
"""
BiLSTM-Attention Network

Benefited from https://github.com/ShawnyXiao/TextClassification-Keras#5-textattbirnn

Implementation of https://arxiv.org/abs/1512.08756
"""


from tensorflow.keras import backend as K
from tensorflow.keras import initializers, regularizers, constraints
from tensorflow.keras.layers import Layer
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Embedding, Dense, Dropout, Bidirectional


In [32]:
from tensorflow.keras import Model
from tensorflow.keras.layers import Embedding, Dense, Bidirectional,LSTM

def sigmoid(x): 
    return 1.0/(1 + np.exp(-x))


class biLSTM(object):
    def __init__(self, maxlen, embedding_dims,
                 class_num=1,
                 last_activation='sigmoid'):
        self.maxlen = maxlen
        self.embedding_dims = embedding_dims
        self.class_num = class_num
        self.last_activation = last_activation

    def get_model(self):
        input = Input((self.maxlen,))
        embedding = createEmbeddingLayer(wordIndex_initial,False)(input) #change
        x = Bidirectional(LSTM(128, return_sequences=False))(embedding)  # LSTM or GRU
        x = Dense(128, activation='relu')(x)

        output = Dense(self.class_num, activation=self.last_activation)(x)
        model = Model(inputs=input, outputs=output)
        return model

#Benefited from https://github.com/ShawnyXiao/TextClassification-Keras#5-textattbirnn
 

In [33]:
def train(X_train_initial,X_test_initial,y_train,y_test) : 
  LSTM_MODEL= biLSTM(50, 100).get_model()

  early_stopping = [EarlyStopping(monitor='val_loss',min_delta=0,restore_best_weights=True, patience=10,verbose=1, mode='auto')]
  LSTM_MODEL.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['acc',f1_m,precision_m, recall_m])
  LSTM_MODEL.summary()

  LSTM_MODEL.fit(X_train_initial, y_train,
            batch_size=16,
            epochs=30,
            callbacks=early_stopping,
            validation_split=0.1)

  loss, accuracy, f1_score, precision, recall = LSTM_MODEL.evaluate(X_train_initial, y_train, verbose=1)
  print(" biLSTM Training Loss: {:.4f}".format(loss))
  print(" biLSTM Accuracy: {:.4f}".format(accuracy))
  print("  biLSTM  f1 score: {:.4f}".format(f1_score))
  print("Attention- BiRNN Precision: {:.4f}".format(precision))
  print("Attention- BiRNN Recall: {:.4f}".format(recall))

  loss, accuracy, f1_score, precision, recall = LSTM_MODEL.evaluate(X_test_initial, y_test, verbose=1)
  print(" biLSTM Test Loss: {:.4f}".format(loss))
  print(" biLSTM Test Accuracy: {:.4f}".format(accuracy))
  print("  biLSTM Test f1 score: {:.4f}".format(f1_score))
  print(" biLSTM  Test Precision: {:.4f}".format(precision))
  print(" biLSTM Test Recall: {:.4f}".format(recall))
 
  probs = LSTM_MODEL.predict(X_test_initial, verbose=1)
  print('lenght of probs : ' ,len(probs))
  predicted_classes=[0 if i < 0.5 else 1 for i in probs]
 

  print(classification_report(y_test, predicted_classes,digits=3))
  return (predicted_classes,probs)





In [34]:
"""
 test  MODE WİTHOUT DOWNSAMPLING
 "tweeter word2vec" 
"""

prediction_attention,probs_attention=train(np.array(X_train_initial),np.array(X_test_initial),np.array(Y_TRAIN_ENCODED_FULL),np.array(Y_TEST_ENCODED_FULL))

missed_words : 5319
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 50)]              0         
                                                                 
 embedding (Embedding)       (None, 50, 300)           6080100   
                                                                 
 bidirectional (Bidirectiona  (None, 256)              439296    
 l)                                                              
                                                                 
 dense (Dense)               (None, 128)               32896     
                                                                 
 dense_1 (Dense)             (None, 1)                 129       
                                                                 
Total params: 6,552,421
Trainable params: 472,321
Non-trainable params: 6,080,100
_________________________

In [35]:
bilstmatt_test_full_result= pd.DataFrame()
bilstmatt_test_full_result['biLSTM-ATTENTION-PREDICTON']=prediction_attention
bilstmatt_test_full_result['biLSTM-ATTENTION-PROBS']=probs_attention
bilstmatt_test_full_result['Gold-label']=Y_TEST_ENCODED_FULL

In [36]:
bilstmatt_test_full_result

,biLSTM-ATTENTION-PREDICTON,biLSTM-ATTENTION-PROBS,Gold-label
0,1,0.592619,1
1,0,0.196643,0
2,0,0.493949,0
3,0,0.021210,0
4,0,0.037849,1
...,...,...,...
855,0,0.139027,1
856,0,0.024650,0
857,1,0.535562,1
858,1,0.710613,0


In [37]:
bilstmatt_test_full_result.to_csv('bilstmatt_test_full_result.csv',header=False)

In [38]:
bilstmatt_valid_full_result= pd.DataFrame()
bilstmatt_valid_full_result['biLSTM-ATTENTION-PREDICTON']=prediction_attention
bilstmatt_valid_full_result['biLSTM-ATTENTION-PROBS']=probs_attention
bilstmatt_valid_full_result['Gold-label']=Y_TEST_ENCODED_FULL
bilstmatt_valid_full_result.to_csv('bilstmatt_valid_full_result.csv',header=False)

In [39]:
bilstmatt_valid_full_result

,biLSTM-ATTENTION-PREDICTON,biLSTM-ATTENTION-PROBS,Gold-label
0,1,0.592619,1
1,0,0.196643,0
2,0,0.493949,0
3,0,0.021210,0
4,0,0.037849,1
...,...,...,...
855,0,0.139027,1
856,0,0.024650,0
857,1,0.535562,1
858,1,0.710613,0
